In [ ]:
import requests
import pandas as pd
from scrapy import Selector

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
    "Accept-Language": "en-US,en;q=0.9"
}

base_url = "https://books.toscrape.com/"

categories = {
    "Poetry": "https://books.toscrape.com/catalogue/category/books/poetry_23/index.html",
    "Mystery": "https://books.toscrape.com/catalogue/category/books/mystery_3/index.html",
    "Travel": "https://books.toscrape.com/catalogue/category/books/travel_2/index.html"
}

all_books = []

for categorie_name, start_url in categories.items():
    print(f"\n Category: {categorie_name} ")

    current_page = start_url

    while current_page:
        response = requests.get(current_page, headers=headers)
        selector = Selector(text=response.text)

        book_links = selector.css("article.product_pod h3 a::attr(href)").getall()

        for link in book_links:
            book_url = requests.compat.urljoin(current_page, link)

            book_response = requests.get(book_url, headers=headers)
            book_sel = Selector(text=book_response.text)

            title = book_sel.css('h1::text').get() or "Unknown title"
            price = book_sel.css('.price_color::text').get() or "£0.00"

            stock_info = book_sel.css('.instock.availability::text').getall()
            stock_status = ''.join(x.strip() for x in stock_info).strip()

            img_src = book_sel.css('div.item.active img::attr(src)').get()
            image_url = requests.compat.urljoin(base_url, img_src) if img_src else None

            breadcrumb = book_sel.css("ul.breadcrumb li a::text").getall()
            category = breadcrumb[-1].strip() if breadcrumb else categorie_name

            rating_raw = book_sel.css("p.star-rating::attr(class)").get()
            rating = rating_raw.split()[-1] if rating_raw else "Unknown"

            all_books.append({
                "title": title,
                "price": price,
                "stock": stock_status,
                "image": image_url,
                "category": category,
                "url": book_url,
                "rating": rating
            })

            print(f" - {title} | {price} | {stock_status} | Rating: {rating}")

        next_page = selector.css("li.next a::attr(href)").get()
        if next_page:
            current_page = requests.compat.urljoin(current_page, next_page)
        else:
            break

df = pd.DataFrame(all_books)
df.to_csv("book_scrap.csv", index=False)
print(df.head())



 Category: Poetry 
 - A Light in the Attic | Â£51.77 | In stock (22 available) | Rating: Three
 - The Black Maria | Â£52.15 | In stock (19 available)In stockIn stockIn stockIn stockIn stockIn stock | Rating: One


KeyboardInterrupt: 